In [ ]:
import os
import sys
project_root = os.getcwd()
sys.path.append(project_root + '/modules')
# Pipeline의 구조를 이해하기 쉽게 하기 위해 작성한 ipynb 파일입니다
from modules.Pipeline import pipe_eval
pipe = pipe_eval(verbose = True) # verbose를 False로 하면 print가 되지 않습니다.
# collection_name = 'chrdb.db',
# persist_directory = 'DB',
# mode = 'NaverCloudEmb'
# topk = 5
# verbose = True : answer과 score을 보여줍니다. 보고싶지 않다면 False로 하면 됩니다.
# default로 지정돼 있습니다. 클래스 선언 때 건네주면 바뀝니다.
pipe.setup(model = 'GPT')
#                   가능한 모델 (default GPT)
#                   1. GPT
#                   2. Exaone
#                   3. Qwen14B
#                   4. Qwen32B
#                   5. HyperClova

In [ ]:
# query = 'SK하이닉스의 2025~2027년 주주환원 정책에서 변경된 고정배당금과 추가 배당금의 활용 방안은 무엇인가요?'
# query = 'NAVER의 분기별 실적 추이 및 전망에 대하여 알려줘'
query = '한화솔루션의 태양광 부문에서 2024년 3분기와 4분기 판매량 감소의 주요 원인은 무엇인가요?'
# query = '로스데렌탈의 업종지수에 대한 비중확대, 중립, 비중축소의 기준은 무엇인가요?'
# validation데이터셋의 첫번째 question
retrieval_result = pipe.Q(query, mode = 'ensemble')
# bm25, dpr, ensemble 가능 (default 'ensemble')


In [3]:
# print(retrieval_result[1].metadata['table'])

In [ ]:
answer = pipe.A(query, retrieval_result)
# query 와 retrieval 결과를 받아 answer을 냅니다.


In [ ]:
len(answer)

In [ ]:
answer = pipe.QA(query, mode = 'ensemble')
# QA를 따로 내는 것이 아니라 한번에 정답을 얻을 수도 있습니다. 이후 정량평가에서는 QA를 사용하면 됩니다.

In [ ]:
len(answer)

In [ ]:
generated_answer, G_retrieval_score, G_generation_score = pipe.QA_eval(mode = 'ensemble', sampling = True)

# validation 데이터셋의 무작위 샘플을 가져와 G-eval로 평가할 수 있습니다.

In [ ]:
eval_df = pipe.QA_eval(mode = 'ensemble', sampling = False) # sampling이 False일 경우, 모든 validation 데이터셋에 대해 평가를 진행합니다.
# 구현이 귀찮아서 일단 ./datas/g_eval_result_{model_name}.csv 경로에 저장되도록 하였습니다..

In [ ]:
import pandas as pd
# retrieval G eval score과 generation G eval score의 점수 부분만 파싱합니다. (저장된 csv에는 전부 문자로 돼 있습니다.)

eval_df['generation_score'] = eval_df['generation_score'].apply(lambda x: x[x.rfind('/')-3:x.rfind('/')])
eval_df['retrieval_score'] = eval_df['retrieval_score'].apply(lambda x: x[x.rfind('/')-3:x.rfind('/')])
eval_df['generation_score'] = pd.to_numeric(eval_df['generation_score'], errors = 'coerce')
eval_df['retrieval_score'] = pd.to_numeric(eval_df['retrieval_score'], errors = 'coerce')

In [ ]:
eval_df['generation_score'].describe()

In [ ]:
retrieval_result[0].metadata['type']

In [8]:
table = """|---------------------|--------|--------|-------|-------|-------|
| 매출액              | 1,829  | 1,749  | 1,906 | 1,706 | 1,936 |
| 영업이익            | 230    | 83     | 106   | 123   | 139   |
| (증감률)            | -58.5  | -63.9  | 27.8  | 15.9  | 13.0  |
| 지배주주순이익      | -      | -      | -     | -     | -     |
| EPS                 | 10.73  | 10.71  | 11.5  | 12.0  | 12.5  |
| PER (H/L)           | 14.5   | 38.9   | 24.3  | 11.5  | 10.5  |
| PBR (H/L)           | 1.40   | 0.78   | 0.73  | -     | -     |
| EV/EBITDA (H/L)     | -      | -      | -     | -     | -     |
| 영업이익률          | -      | -      | -     | -     | -     |
| ROE                 | -      | -      | -     | -     | -     |"""

In [11]:
import pandas as pd
df = pd.read_csv('/data/ephemeral/home/clone/src/modules/datas/SK케미칼_DB금융투자(2024.11.06)/SK케미칼_DB금융투자(2024.11.06).csv')

In [21]:
import pandas as pd
import re

def markdown_to_df(markdown_table: str) -> pd.DataFrame:
    """
    마크다운 테이블을 Pandas DataFrame으로 변환하는 함수
    
    :param markdown_table: 마크다운 형식의 테이블 문자열
    :return: 변환된 Pandas DataFrame
    """
    lines = markdown_table.strip().split("\n")
    
    # 헤더 추출
    headers = [h.strip() for h in lines[0].split("|") if h.strip()]
    
    # 데이터 행 추출 (두 번째 줄은 구분선이므로 제외)
    data_rows = lines[2:]
    
    # 데이터 처리
    data = []
    for row in data_rows:
        values = [v.strip() for v in row.split("|") if v.strip()]
        data.append(values)
    
    # 데이터프레임 생성
    df = pd.DataFrame(data, columns=headers)
    return df

In [28]:
df = pd.read_csv('/data/ephemeral/home/clone/src/modules/datas/SK케미칼_DB금융투자(2024.11.06)/SK케미칼_DB금융투자(2024.11.06).csv')



In [ ]:
q = 'SK하이닉스의 2024년 핵심사업에 대해 알려줘.'
result = pipe.Q(query)

In [ ]:
for i in result:
    print(i.metadata['type'])

In [ ]:
print(answer1)

In [ ]:
import os
import sys
project_root = os.getcwd()
sys.path.append(project_root + '/modules')
from Pipeline import pipe_service

pipe = pipe_service()
pipe.Q('SK하이닉스의 핵심사업에 대해 알려줘.')

